## Narrowing down to probes of interest

### imports

In [4]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer  # For imputation in Method 4a

## Narrowing down to probes of interest

### setup env for multithreading

In [5]:
os.environ["OMP_NUM_THREADS"] = "16"
os.environ["OPENBLAS_NUM_THREADS"] = "16"
os.environ["MKL_NUM_THREADS"] = "16"
os.environ["VECLIB_MAXIMUM_THREADS"] = "16"
os.environ["NUMEXPR_NUM_THREADS"] = "16"

### Data Loading and Preparation

In [9]:
pcawg_prim_window_meth_df_merged = pd.read_csv("../_OUTPUTS_/merged_sith_meth_pcawg_prim_window.tsv", sep = "\t")
pcawg_sith_corr_meth_df_merged = pd.read_csv("../_OUTPUTS_/merged_sith_meth_pcawg_sith_corr.tsv", sep = "\t")
pcawg_iqr_corr_meth_df_merged = pd.read_csv("../_OUTPUTS_/merged_sith_meth_pcawg_iqr_corr.tsv", sep = "\t")

In [12]:
# Define bins and labels for the sith_level column
bins = [-float('inf'), 0.7, 0.8, float('inf')]
labels = ['<0.7', '0.7-0.8', '>0.8']

# Create a list of datasets and key parameters.
# Note: For the IQR dataset we sort on the 'INT_IQR' column, but for the others we sort on 'SITH'.
datasets = [
    {
        'name': 'pcawg_prim_window',
        'df': pcawg_prim_window_meth_df_merged,
        'sort_col': 'SITH'
    },
    {
        'name': 'pcawg_sith_corr',
        'df': pcawg_sith_corr_meth_df_merged,
        'sort_col': 'SITH'
    },
    {
        'name': 'pcawg_iqr_corr',
        'df': pcawg_iqr_corr_meth_df_merged,
        'sort_col': 'INT_IQR'
    }
]

# A list to gather all result rows
results_list = []

# Loop over each dataset and perform the pipeline of analyses.
for dset in datasets:
    name = dset['name']
    df = dset['df']
    sort_col = dset['sort_col']
    
    # --- Step 1: Create the 'sith_level' column ---
    df['sith_level'] = pd.cut(df['SITH'], bins=bins, labels=labels)

    # --- Step 2: Sort the DataFrame ---
    sorted_df = df.sort_values(sort_col, ascending=False).reset_index(drop=True)

    # --- Step 3: Extract the probe columns (columns starting with 'cg') ---
    probe_cols = [col for col in sorted_df.columns if col.startswith('cg')]
    probe_cols = list(probe_cols)
    data_matrix = sorted_df[probe_cols]

    # --- Step 4: Filter probes based on initial conditions ---
    # Remove probes where all values are ≤ 0.5 or all values are ≥ 0.5
    mask = (data_matrix > 0.5).any(axis=0) & (data_matrix < 0.5).any(axis=0)
    probe_cols = data_matrix.columns[mask]
    probe_cols = list(probe_cols)
    data_matrix = data_matrix[probe_cols]

    # Common parameters
    N = 10  # number of top probes to report

    # =============================================================================
    # Method 1: Select Probes with the Highest Variance
    # =============================================================================
    probe_variances = data_matrix.var(axis=0)
    top_variance_probes = probe_variances.nlargest(N).index.tolist()
    for rank, probe in enumerate(top_variance_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'var_1',
            'probe': probe,
            'variance': probe_variances[probe],
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 2: Select Probes with the Highest Correlation to SITH Score
    # =============================================================================
    correlations = data_matrix.apply(lambda x: x.corr(sorted_df['SITH']), axis=0)
    top_correlated_probes = correlations.abs().nlargest(N).index.tolist()
    for rank, probe in enumerate(top_correlated_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'corr_2',
            'probe': probe,
            'variance': np.nan,
            'correlation': correlations[probe],
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 3: Perform Statistical Tests (ANOVA) Between Groups
    # =============================================================================
    data_matrix_with_labels = data_matrix.copy()
    data_matrix_with_labels.reset_index(drop=True, inplace=True)
    sorted_df.reset_index(drop=True, inplace=True)
    data_matrix_with_labels['sith_level'] = sorted_df['sith_level']
    subset_cols = ['sith_level'] + probe_cols
    data_matrix_with_labels.dropna(subset=subset_cols, inplace=True)

    p_values = {}
    for probe in probe_cols:
        groups = []
        for level in data_matrix_with_labels['sith_level'].unique():
            group = data_matrix_with_labels[data_matrix_with_labels['sith_level'] == level][probe]
            groups.append(group)
        if len(groups) >= 2:
            stat, p_value = f_oneway(*groups)
            p_values[probe] = p_value

    p_values_series = pd.Series(p_values).dropna()
    top_pvalue_probes = p_values_series.nsmallest(N).index.tolist()
    for rank, probe in enumerate(top_pvalue_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'anova_3',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': p_values[probe],
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 4a: Feature Selection Using Random Forest (with Imputation)
    # =============================================================================
    X = data_matrix.copy()
    y = sorted_df['SITH']
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    rf_impute = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_impute.fit(X_imputed, y)
    importances_impute = pd.Series(rf_impute.feature_importances_, index=X_imputed.columns)
    top_importance_probes_impute = importances_impute.nlargest(N).index.tolist()
    for rank, probe in enumerate(top_importance_probes_impute, start=1):
        results_list.append({
            'dataset': name,
            'method': 'rf_impute_4a',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': importances_impute[probe],
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 4b: Feature Selection Using Random Forest (dropping rows with NaNs)
    # =============================================================================
    X = data_matrix.copy()
    y = sorted_df['SITH']
    data_combined = X.copy()
    data_combined['SITH'] = y
    data_combined.dropna(inplace=True)
    X_dropped = data_combined.drop(columns=['SITH'])
    y_dropped = data_combined['SITH']

    rf_drop = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_drop.fit(X_dropped, y_dropped)
    importances_drop = pd.Series(rf_drop.feature_importances_, index=X_dropped.columns)
    top_importance_probes_drop = importances_drop.nlargest(N).index.tolist()
    for rank, probe in enumerate(top_importance_probes_drop, start=1):
        results_list.append({
            'dataset': name,
            'method': 'rf_drop_4b',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': importances_drop[probe],
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 5: Common Probes of Interest
    # =============================================================================
    probes_of_interest = [
        'cg00870279','cg01842321','cg01861555','cg06805320','cg07235253','cg08327371',
        'cg08532569','cg08598483','cg10982913','cg12164232','cg14387626','cg16272777',
        'cg16338877','cg16401270','cg17163967','cg19477190','cg21142743','cg22635155',
        'cg27086014'
    ]
    # Keep only those that passed the filtering
    probes_of_interest = [p for p in probes_of_interest if p in probe_cols]
    for rank, probe in enumerate(probes_of_interest, start=1):
        results_list.append({
            'dataset': name,
            'method': 'common_5',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 6: Combine Multiple Criteria (variance + ANOVA p-value)
    # =============================================================================
    top_50_variance_probes = probe_variances.nlargest(50).index.tolist()
    p_values_combined = {}
    for probe in top_50_variance_probes:
        groups = []
        for level in sorted_df['sith_level'].unique():
            group = sorted_df[sorted_df['sith_level'] == level][probe]
            groups.append(group)
        if len(groups) >= 2:
            stat, p_val = f_oneway(*groups)
            p_values_combined[probe] = p_val

    top_combined_probes = pd.Series(p_values_combined).nsmallest(N).index.tolist()
    for rank, probe in enumerate(top_combined_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'combined_6',
            'probe': probe,
            'variance': probe_variances[probe],
            'correlation': np.nan,
            'p_value': p_values_combined[probe],
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 7: Clustering and Selecting Representative Probes
    # =============================================================================
    X_transposed = data_matrix.T.dropna(how='all')  # probes as rows
    X_transposed_imputed = X_transposed.apply(lambda row: row.fillna(row.mean()), axis=1)
    X_transposed_imputed = X_transposed_imputed.dropna(how='any', axis=0)
    n_clusters = 10
    clustering = AgglomerativeClustering(n_clusters=n_clusters)
    cluster_labels = clustering.fit_predict(X_transposed_imputed)
    clustered_probes = pd.DataFrame({'probe': X_transposed_imputed.index, 'cluster': cluster_labels})
    selected_probes = clustered_probes.groupby('cluster')['probe'].first().tolist()
    
    # For each cluster's "representative" probe
    for clust, probe in zip(sorted(clustered_probes['cluster'].unique()), selected_probes):
        results_list.append({
            'dataset': name,
            'method': 'cluster_7',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': clust
        })

    # =============================================================================
    # Method 8: Filtering Based on Methylation Range
    # =============================================================================
    probe_ranges = data_matrix.max(axis=0) - data_matrix.min(axis=0)
    top_range_probes = probe_ranges.nlargest(N).index.tolist()
    for rank, probe in enumerate(top_range_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'range_8',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': probe_ranges[probe],
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 9: Filtering Based on Specific Thresholds
    # =============================================================================
    high_threshold = 0.8
    low_threshold = 0.2
    high_proportion = (data_matrix > high_threshold).mean(axis=0)
    low_proportion = (data_matrix < low_threshold).mean(axis=0)
    selected_probes_threshold = high_proportion[high_proportion > 0.3].index.intersection(
                                low_proportion[low_proportion > 0.3].index).tolist()
    selected_probes_threshold = selected_probes_threshold[:N]  # limit to top N
    for rank, probe in enumerate(selected_probes_threshold, start=1):
        results_list.append({
            'dataset': name,
            'method': 'threshold_9',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': np.nan,
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': high_proportion[probe],
            'prop_below_0.2': low_proportion[probe],
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 10: Differential Methylation Analysis Using Linear Regression
    # =============================================================================
    data_for_analysis = data_matrix.copy()
    data_for_analysis.reset_index(drop=True, inplace=True)
    sorted_df.reset_index(drop=True, inplace=True)
    data_for_analysis['sith_level'] = sorted_df['sith_level']
    subset_cols = ['sith_level'] + probe_cols
    data_for_analysis.dropna(subset=subset_cols, inplace=True)
    # Convert categorical level to numeric codes
    data_for_analysis['sith_level_code'] = data_for_analysis['sith_level'].cat.codes
    
    p_values_differential = {}
    for probe in probe_cols:
        formula = f"Q('{probe}') ~ sith_level_code"
        try:
            model = smf.ols(formula, data=data_for_analysis).fit()
            p_values_differential[probe] = model.pvalues['sith_level_code']
        except Exception:
            continue
    
    p_values_differential = {k: v for k, v in p_values_differential.items() if not pd.isnull(v)}
    top_differential_probes = pd.Series(p_values_differential).nsmallest(N).index.tolist()
    for rank, probe in enumerate(top_differential_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'diffmeth_10',
            'probe': probe,
            'variance': np.nan,
            'correlation': np.nan,
            'p_value': p_values_differential[probe],
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': np.nan,
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': np.nan,
            'cluster_label': np.nan
        })

    # =============================================================================
    # Method 11: Visualizing All Filters with an Aggregate Score
    # =============================================================================
    criteria_df = pd.DataFrame(index=probe_cols)
    criteria_df['variance'] = probe_variances
    criteria_df['correlation'] = correlations.abs()
    criteria_df['range'] = probe_ranges
    # Insert ANOVA p_values, fill missing with 1.0 so they don't artificially rank well
    criteria_df['p_value'] = p_values_series.reindex(probe_cols).fillna(1.0)

    # Normalize columns for comparability
    scaler = MinMaxScaler()
    subset_for_scaling = criteria_df[['variance', 'correlation', 'range', 'p_value']].fillna(0)
    criteria_normalized = pd.DataFrame(
        scaler.fit_transform(subset_for_scaling),
        index=subset_for_scaling.index,
        columns=['variance', 'correlation', 'range', 'p_value']
    )
    # Invert the p_value scale so that lower p-values score higher
    criteria_normalized['p_value'] = 1 - criteria_normalized['p_value']
    criteria_normalized['aggregate_score'] = criteria_normalized.mean(axis=1)

    top_aggregate_probes = criteria_normalized['aggregate_score'].nlargest(N).index.tolist()
    for rank, probe in enumerate(top_aggregate_probes, start=1):
        results_list.append({
            'dataset': name,
            'method': 'aggregate_11',
            'probe': probe,
            'variance': criteria_df.loc[probe, 'variance'],
            'correlation': criteria_df.loc[probe, 'correlation'],
            'p_value': criteria_df.loc[probe, 'p_value'],
            'rf_importance_impute': np.nan,
            'rf_importance_drop': np.nan,
            'range': criteria_df.loc[probe, 'range'],
            'prop_above_0.8': np.nan,
            'prop_below_0.2': np.nan,
            'aggregate_score': criteria_normalized.loc[probe, 'aggregate_score'],
            'cluster_label': np.nan
        })

# -------------------------------------------------------------------------
# Create a final results DataFrame from our accumulated rows and save to CSV
# -------------------------------------------------------------------------
results_df = pd.DataFrame(results_list)
results_df.to_csv('../_OUTPUTS_/pcawg_probes_of_interest.csv', index=False)

print("Done! Results saved to '../_OUTPUTS_/pcawg_probes_of_interest.csv'.")

Done! Results saved to '../_OUTPUTS_/pcawg_probes_of_interest.csv'.


In [13]:
results_df

,dataset,method,probe,variance,correlation,p_value,rf_importance_impute,rf_importance_drop,range,prop_above_0.8,prop_below_0.2,aggregate_score,cluster_label
0,pcawg_prim_window,var_1,cg00840341,0.047657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pcawg_prim_window,var_1,cg13363969,0.043906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pcawg_prim_window,var_1,cg15992272,0.043533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pcawg_prim_window,var_1,cg15618210,0.043486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pcawg_prim_window,var_1,cg08566882,0.043257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,pcawg_iqr_corr,aggregate_11,cg14166284,0.030293,0.489655,0.001336,NaN,NaN,0.777561,NaN,NaN,0.733290,NaN
328,pcawg_iqr_corr,aggregate_11,cg19295951,0.046240,0.330906,0.000072,NaN,NaN,0.867622,NaN,NaN,0.732978,NaN
329,pcawg_iqr_corr,aggregate_11,cg23882019,0.028151,0.444167,0.000005,NaN,NaN,0.823488,NaN,NaN,0.723090,NaN
330,pcawg_iqr_corr,aggregate_11,cg08328777,0.044698,0.307794,0.000038,NaN,NaN,0.880604,NaN,NaN,0.722356,NaN
